# IMPORTS

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import logging
import typing
import gc

In [ ]:
# if you are running this notebook from 'home/dev/enefit/notebook'. 
os.chdir('..') # else adjust to point to the root of the project.

In [ ]:
%%capture output

%load_ext kedro.ipython
%reload_kedro

if 'output' in locals() and 'error' in output.stderr:
    output.show()